In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import scipy.io
import scipy.signal
from einops import reduce, rearrange
import numpy as np

In [2]:
from torch_geometric.data import InMemoryDataset, Data, DataLoader
from Electrodes import Electrodes
from tqdm import tqdm
class DEAPDatasetEEGFeatures(InMemoryDataset):
    
  def __init__(self, root, raw_dir,processed_dir, transform=None, pre_transform=None,include_edge_attr = False, undirected_graphs = True, add_global_connections=True, participant_from=1, participant_to=32,window_size=128, n_videos=40):
      self._raw_dir = raw_dir
      self._processed_dir = processed_dir
      self.participant_from = participant_from
      self.participant_to = participant_to
      self.n_videos = n_videos
      self.window_size = window_size
      # Whether or not to include edge_attr in the dataset
      self.include_edge_attr = include_edge_attr
      # If true there will be 1024 links as opposed to 528
      self.undirected_graphs = undirected_graphs
      # Instantiate class to handle electrode positions
      print('Using global connections' if add_global_connections else 'Not using global connections')
      self.electrodes = Electrodes(add_global_connections, expand_3d = False)
      super(DEAPDatasetEEGFeatures, self).__init__(root, transform, pre_transform)
      self.data, self.slices = torch.load(self.processed_paths[0])
      

  @property
  def raw_dir(self):
      return f'{self.root}/{self._raw_dir}'

  @property
  def processed_dir(self):
      return f'{self.root}/{self._processed_dir}'

  @property
  def raw_file_names(self):
      raw_names = [f for f in os.listdir(self.raw_dir)]
      raw_names.sort()
      return raw_names

  @property
  def processed_file_names(self):
      if not os.path.exists(self.processed_dir):
        os.makedirs(self.processed_dir)
      file_name = f'{self.participant_from}-{self.participant_to}' if self.participant_from is not self.participant_to else f'{self.participant_from}'
      return [f'deap_processed_graph.{file_name}.dataset']

  def process(self):
        # Number of nodes per graph
        n_nodes = len(self.electrodes.positions_3d)

        if self.undirected_graphs:
            source_nodes, target_nodes = np.repeat(np.arange(0,n_nodes),n_nodes), np.tile(np.arange(0,n_nodes),n_nodes)
        else:
            source_nodes, target_nodes = np.tril_indices(n_nodes,n_nodes)
        
        edge_attr = self.electrodes.adjacency_matrix[source_nodes,target_nodes]
        
        # Remove zero weight links
        mask = np.ma.masked_not_equal(edge_attr, 0).mask
        edge_attr,source_nodes,target_nodes = edge_attr[mask], source_nodes[mask], target_nodes[mask]

        edge_attr, edge_index = torch.FloatTensor(edge_attr), torch.tensor([source_nodes,target_nodes], dtype=torch.long)
        
        # Expand edge_index and edge_attr to match windows
        e_edge_index = edge_index.clone()
        e_edge_attr = edge_attr.clone()
        for i in range(128*60//self.window_size-1):
            a = edge_index + e_edge_index.max() + 1
            e_edge_index = torch.cat([e_edge_index,a],dim=1)
            e_edge_attr = torch.cat([e_edge_attr,edge_attr],dim=0)

        # List of graphs that will be written to file
        data_list = []
        pbar = tqdm(range(self.participant_from,self.participant_to+1))
        for participant_id in pbar:
            raw_name = [e for e in self.raw_file_names if str(participant_id).zfill(2) in e][0]
            pbar.set_description(raw_name)
            # Load raw file as np array
            participant_data = scipy.io.loadmat(f'{self.raw_dir}/{raw_name}')
            signal_data = torch.FloatTensor(participant_data['data'][:,:32,128*3:])
            processed = []
            for i, video in enumerate(signal_data[:self.n_videos,:,:]):
                video = video.reshape(-1,128)
                # Power spectral density for each channel
                psd = scipy.signal.periodogram(video)[1]
                # Should we add MinMax/Z scaler?
                data = Data(x=torch.FloatTensor(psd),edge_attr=e_edge_attr,edge_index=e_edge_index, y=torch.FloatTensor([participant_data['labels'][i]])) if self.include_edge_attr else Data(x=torch.FloatTensor(psd), edge_index=e_edge_index, y=torch.FloatTensor([participant_data['labels'][i]]))
                data_list.append(data) 
               
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

In [4]:
# Constants used to define data paths
ROOT_DIR = './'
RAW_DIR = 'data/matlabPREPROCESSED'
PROCESSED_DIR = 'data/graphProcessedData'

dataset = DEAPDatasetEEGFeatures(root= ROOT_DIR, raw_dir= RAW_DIR, processed_dir= PROCESSED_DIR)
# Subject-independent classification
dataset = dataset.shuffle()
dataset[0]

s01.mat:   0%|          | 0/32 [00:00<?, ?it/s]

Using global connections
Processing...


s32.mat: 100%|██████████| 32/32 [00:19<00:00,  1.63it/s]


Done!


Data(edge_index=[2, 11640], x=[1920, 65], y=[1, 4])

In [5]:
# 70% used for training, 15% validation and 15% testing
splt_idx = (int(0.7*len(dataset)),int(0.85*len(dataset)))

train_dataset = dataset[:splt_idx[0]]
val_dataset = dataset[splt_idx[0]:splt_idx[1]]
test_dataset = dataset[splt_idx[1]:]

train_dataset,val_dataset,test_dataset

(DEAPDatasetEEGFeatures(896),
 DEAPDatasetEEGFeatures(192),
 DEAPDatasetEEGFeatures(192))

In [88]:
BS = 16
train_loader = DataLoader(train_dataset, batch_size=BS)
val_loader = DataLoader(val_dataset, batch_size=BS)
test_loader = DataLoader(test_dataset, batch_size=BS)

In [89]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')

In [90]:
from torch_geometric.nn import GCNConv
class Model(torch.nn.Module):
    def __init__(self, in_channels):
        super(Model, self).__init__()
        self.gconv1 = GCNConv(in_channels, 256, aggr='add')
        self.gconv2 = GCNConv(256, 128, aggr='add')
        self.gconv3 = GCNConv(128, 1, aggr='add')
        self.gconv4 = GCNConv(8, 1, aggr='add')
        
        self.conv1 = nn.Conv1d(60, 8, 1, stride=1)
        
#         .to(device)
        self.lin1 = nn.Linear(256,32)
        self.lin2 = nn.Linear(32,1)
        
    def forward(self, batch):
        bs = len(torch.unique(batch.batch))
        x, edge_index = batch.x, batch.edge_index
        
#         print(x)
        
        x = self.gconv1(x, edge_index)
#         print(x)
        x = torch.tanh(x)
        if x[0][0].isnan():
            raise 'err'
        x = self.gconv2(x, edge_index)
        x = torch.tanh(x)
        x = self.gconv3(x, edge_index)
        x = torch.tanh(x)
        x = F.dropout(x, p=0.2, training=self.training)
#         x = self.gconv4(x, edge_index)
#         x = torch.tanh(x)
#         print(x)
        x = x.reshape(bs,-1,32)
#         x.relu()
        
        x = self.conv1(x)
        x = torch.relu(x)
        x = x.reshape(bs,-1)
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.lin1(x)
        x.sigmoid()
        x = self.lin2(x)
#         print(x)
#         raise 'err'
        return x.view(-1)

In [92]:
model = Model(dataset[0].x[1].shape[0])     
pytorch_total_params = sum(p.numel() for p in model.parameters())
print(f'Model parameter count: {pytorch_total_params}')

# optimizer = torch.optim.Adadelta(model.parameters(), lr=.1, rho=0.9, eps=1e-06, weight_decay=1e-5)
# optimizer = torch.optim.SGD(model.parameters(),lr=1e-1, weight_decay=1e-3)
optimizer = torch.optim.Adam(model.parameters(), lr=.001)
# optimizer = torch.optim.RMSprop(model.parameters(), lr=0.01, weight_decay=1e-6)

criterion = nn.MSELoss()

model = model.to(device)

def train():
    model.train()
    losses = []
    right = 0
    tot = 0
    for batch in train_loader:
        optimizer.zero_grad()
        batch = batch.to(device)
        y = batch.y[:,0] # Valence
        out = model(batch)
        loss = criterion(out,y)
        loss.backward()
        losses.append(loss.item())
        optimizer.step()
        right += torch.eq(out > 5, y > 5).sum().item()
        tot += y.shape[0]
    return np.array(losses).mean(), right/tot

def test(loader,verbose=False):
    model.eval()
    losses = []
    right = 0
    tot = 0
    for batch in loader:
        batch = batch.to(device)
        y = batch.y[:,0] # Valence
        out = model(batch)
        loss = criterion(out,y)
        losses.append(loss.item())
        right += torch.eq(out > 5, y > 5).sum().item()
        tot += y.shape[0]
    return np.array(losses).mean(), right/tot

best_val_loss = np.inf
esp = 0
MAX_ESP = 50
for epoch in range(1, 1000):
    train_loss, train_acc = train()
    val_loss, val_acc = test(val_loader)
#     if val_loss <= best_val_loss:
#         best_val_loss = val_loss
#         esp = 0
#     else:
#         esp += 1
#         if esp >= MAX_ESP:
#             break
#     if epoch%5 == 0:
    print(f'Epoch {epoch} ;t loss: {train_loss:.5f} ;t acc: {train_acc:.2f} ;v loss: {val_loss:.5f} ;v acc: {val_acc:.2f}')

print('Finished training')

Model parameter count: 58675
Epoch 1 ;t loss: 8.88811 ;t acc: 0.49 ;v loss: 4.20414 ;v acc: 0.40
Epoch 2 ;t loss: 4.82150 ;t acc: 0.53 ;v loss: 4.05339 ;v acc: 0.60
Epoch 3 ;t loss: 4.83002 ;t acc: 0.53 ;v loss: 4.16717 ;v acc: 0.40
Epoch 4 ;t loss: 4.86645 ;t acc: 0.51 ;v loss: 4.25214 ;v acc: 0.40
Epoch 5 ;t loss: 4.88422 ;t acc: 0.50 ;v loss: 4.30437 ;v acc: 0.40
Epoch 6 ;t loss: 4.89220 ;t acc: 0.50 ;v loss: 4.33029 ;v acc: 0.40
Epoch 7 ;t loss: 4.89549 ;t acc: 0.50 ;v loss: 4.33893 ;v acc: 0.40
Epoch 8 ;t loss: 4.89643 ;t acc: 0.50 ;v loss: 4.33632 ;v acc: 0.40
Epoch 9 ;t loss: 4.89592 ;t acc: 0.50 ;v loss: 4.32627 ;v acc: 0.40
Epoch 10 ;t loss: 4.89453 ;t acc: 0.50 ;v loss: 4.31151 ;v acc: 0.40
Epoch 11 ;t loss: 4.89262 ;t acc: 0.50 ;v loss: 4.29276 ;v acc: 0.40
Epoch 12 ;t loss: 4.88954 ;t acc: 0.50 ;v loss: 4.27284 ;v acc: 0.40
Epoch 13 ;t loss: 4.88601 ;t acc: 0.50 ;v loss: 4.25128 ;v acc: 0.39
Epoch 14 ;t loss: 4.88216 ;t acc: 0.50 ;v loss: 4.23051 ;v acc: 0.40
Epoch 15 ;t lo

KeyboardInterrupt: 

In [93]:
loss, acc = test(test_loader)
print(f'Test loss: {loss} ; Test acc: {acc}')

# TODO: esp, kfold val , scheduler(?)

Test loss: 4.422489643096924 ; Test acc: 0.578125
